## Predict Performance Scores based on Number of Days taken to Submit Assignments

This project illustrates the application of the K-Nearest Neighbor Machine Learning algorithm in its simplest form. It predicts the most likely score based on how long it takes for an assignment to be submitted by a learner. Given this knowledge, Educators and policy makers can influence how, when, and why students should work at a certain pace. Further, given this information, numbers of newly qualified people can be predicted and companies can adjust workforce strategy accordingly.

This the [Open Learning Analytics dataset](https://analyse.kmi.open.ac.uk/open_dataset) used in this project

In [1]:
import pandas as pd # for this analysis, pandas is more suitable than a regular python list
import numpy as np # required to calculate Euclidean distances, absolute values, etc

In [2]:
# Read the data into a dataframe

assessments = pd.read_csv("C:\dataHub\studentAssessment.csv") 
print(assessments.sample(n = 5)) # dynamically select a sample, provides snapshots of the data

        id_assessment  id_student  date_submitted  is_banked  score
111803          34878      605538              17          0     84
21255           25359      619297             157          0     55
119265          34865      510186              32          0     86
26391           34874      599957              47          0     58
100956          25334      291133              19          0     81


Initial plan: Find a few SIMILAR submission days, calculate the average test score of the submission days, and then determine the average test score expected for any submission day value. Try the k-nearest neigbor algorithm first since its functionality most closely fits my requirement

To calculate the (Euclidean) distance, use the date_submitted feature from the data set

In [3]:
# Calculate the distance between the chosen number of submission days and whatever the submission day value is 
# I chose 300 days, which is about half of the maximum 608 days in the data set

our_sub_day = 300
first_sub_day = assessments.iloc[0]['date_submitted']
first_euc_distance = np.abs(first_sub_day - our_sub_day)
print(first_sub_day)
print(our_sub_day)
print(first_euc_distance)

248
300
52


In [4]:
# Calculate the distance between each 'date_submitted' value and 300, our chosen number of days to submit
# This requires calculating absolute values and returning a series of values in the new 'distance' field

assessments['distance'] = assessments['date_submitted'].apply(lambda x: np.abs(x - our_sub_day))
print(assessments['distance'].value_counts().head())

282    2747
231    2744
281    2475
100    2429
279    2281
Name: distance, dtype: int64


In [5]:
# Checking our data with the new distance values

print(assessments.sample(n = 5))

        id_assessment  id_student  date_submitted  is_banked  score  distance
136252          34888      242998              85          0     91       215
61595           24287      630449              69          0     72       231
129506          30709      603222              31          0     89       269
53316           34896      633983             226          0     70        74
39080           34883      502249             196          0     64       104


In [6]:
# To prevent bias, randomize the dataset

assessments = assessments.loc[np.random.permutation(len(assessments))] # Randomize index values and shuffle the dataset
assessments = assessments.sort_values('distance') # Sort by the 'distance' field
print(assessments.iloc[0:4]['score']) # display scores of the 5 MOST SIMILAR ASSESSMENTS

140325    93
120037    87
101116    82
50816     69
Name: score, dtype: int64


In [7]:
# Calculate the average score of the 4 MOST SIMILAR ASSESSMENTS
avg_score = assessments.iloc[:4]['score'].mean()
print(avg_score)

82.75


Based on applying the k-nearest neighbor algorithm, a student who submits an assignment halfway into the
allocated time (300th day out of a possible 608 days) is predicated to score 89.8%

Next, we determine the most likely score given other submission days. This entails determining how good this evaluation is.

In [8]:
# Construct a function that uses the k-nearest neighbor algorithm to predict a score for a given submission day

def predict_score(new_submission): # number of days when assignment is submitted
    temp_assess = assessments.copy() # assign current data to a new df
    
    # calculate absolute distance between newly submitted value (days) and each current days value
    temp_assess['distance'] = temp_assess['date_submitted'].apply(lambda x: np.abs(x - new_submission))
    temp_assess = temp_assess.sort_values('distance') # sort by newly calculated 'distance' values
    nearest_neighbors = temp_assess.iloc[0:4]['score'] # select 4 nearest neighbors' scores
    predicted_score = nearest_neighbors.mean() # calculate the 4 nearest neighbors' average score
    return(predicted_score) # the average score is the return value for the predict_score function

# Use the predict_score function to suggest a score based on the number of days an assignment is submitted

submit_after_100_days = predict_score(100)
submit_after_300_days = predict_score(400)
submit_after_500_days = predict_score(500)

print(submit_after_100_days)
print(submit_after_300_days)
print(submit_after_500_days)

80.25
79.0
84.25


In [9]:
# For this univariate exercise, a single feature and a fixed k value was used. The simplicity helps with
# highlighting Machine Learning's K-Nearest Neighbor algorithm workflow.

## Conclusion

In this project, I illustrated an application of the K-Nearest Neighbor Machine Learning algorithm in its simplest form. "Simple" because the analysis is univariate and the k value is fixed. A more robust application of this ML workflow on similar learning analytics data would, for instance, consider more than one feature. One would expect that features such as disability status, highest education, and the number of previous attempts would predict performance differently. 

I concluded that based on submitting an assignment roughly halfway into the allocated period, a score of 89.8% can be predicted. This work is not complete; for instance, a suggested next step is to evaluate how good this prediction is. 

Finally, please note that this ML method can be a computational nightmare if working with very large data sets: each time you need to predict a score based on a new days value submitted, calculating distances and ranking occurs on the entire data set. It is therefore recommended that it be applied in smaller data sets for purposes of testing the code.